---
title: "Airgead don Todchaí"
format: html
theme:
  light: flatly
jupyter: python3
params:
  starting_salary: 32000
  salary_increase_rate_early: 0.10
  salary_increase_rate_late: 0.02
  pension_contribution_rate: 0.10
  increase_contribution_rate: 0.60
  investment_return: 0.07
  fee_rate: 0.01
  years: 30
---

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

# Use parameters from Quarto's YAML header
def pension_fund_simulation(params):
    starting_salary = params['starting_salary']
    salary_increase_rate_early = params['salary_increase_rate_early']
    salary_increase_rate_late = params['salary_increase_rate_late']
    pension_contribution_rate = params['pension_contribution_rate']
    increase_contribution_rate = params['increase_contribution_rate']
    investment_return = params['investment_return']
    fee_rate = params['fee_rate']
    years = params['years']

    # Initialize lists
    years_list, salary_list, annual_contribution_list = [], [], []
    pension_balance_list, pension_balance_fixed_list = [], []
    fees_accumulated_list, pension_after_fees_list, pension_after_fees_fixed_list = [], [], []

    # Initial conditions
    salary = starting_salary
    pension_balance = pension_contribution_rate * salary
    pension_balance_fixed = pension_contribution_rate * salary
    annual_contribution = pension_contribution_rate * salary
    total_fees_accumulated = 0

    for year in range(1, years + 1):
        if year <= 5:
            salary_increase = salary * salary_increase_rate_early
            additional_contribution = salary_increase * increase_contribution_rate
            annual_contribution += additional_contribution
            salary *= (1 + salary_increase_rate_early)
        else:
            salary *= (1 + salary_increase_rate_late)
            annual_contribution = max(annual_contribution, salary * pension_contribution_rate)

        annual_contribution_fixed = salary * pension_contribution_rate

        pension_balance = (pension_balance + annual_contribution) * (1 + investment_return)
        pension_balance_fixed = (pension_balance_fixed + annual_contribution_fixed) * (1 + investment_return)

        fees_paid = pension_balance * fee_rate
        pension_balance_after_fees = pension_balance - fees_paid
        total_fees_accumulated += fees_paid

        fees_paid_fixed = pension_balance_fixed * fee_rate
        pension_balance_after_fees_fixed = pension_balance_fixed - fees_paid_fixed

        # Store results
        years_list.append(year)
        salary_list.append(salary)
        annual_contribution_list.append(annual_contribution)
        pension_balance_list.append(pension_balance)
        pension_balance_fixed_list.append(pension_balance_fixed)
        fees_accumulated_list.append(total_fees_accumulated)
        pension_after_fees_list.append(pension_balance_after_fees)
        pension_after_fees_fixed_list.append(pension_balance_after_fees_fixed)

    # Create DataFrame
    df = pd.DataFrame({
        "Year": years_list,
        "Salary (€)": salary_list,
        "Annual Contribution (€)": annual_contribution_list,
        "Pension Balance After Fees (€1,000s) (Increased Contributions)": np.array(pension_after_fees_list)/1000,
        "Pension Balance After Fees (€1,000s) (Fixed Contributions)": np.array(pension_after_fees_fixed_list)/1000,
        "Total Fees Earned (€1,000s)": np.array(fees_accumulated_list)/1000,
    })

    # Plot interactive chart
    fig = px.line(df, x="Year",
                  y=["Pension Balance After Fees (€1,000s) (Increased Contributions)",
                     "Pension Balance After Fees (€1,000s) (Fixed Contributions)",
                     "Total Fees Earned (€1,000s)"],
                  labels={"value": "Amount (€1,000s)", "Year": "Years"},
                  title="Pension Balance Growth Comparison: Increased vs Fixed Contributions")

    fig.update_layout(yaxis_tickformat=",")
    fig.show()

# Run the function
pension_fund_simulation(params)